### Edit this so that it's the month you want to generate:

In [18]:
year = 2022
month = 12

# Do you only want the severe events?
severe = True

### Now hit the double right arrows in the menu above. The rest of this works on its own.

In [19]:
import pandas as pd
import csv
from re import search, sub
from os import listdir 

def fill_or_swap_duplicates(final_df, new_df, column='nws_id'):
    if len(final_df) == 0:
        # If the final df is empty, fill it with the file's contents
        return new_df
    else:
        # Otherwise, remove all column values the final df has in common with the new file
        # and insert the newer values (assumes the files are in order).
        # This assumes the files are in alphabetical order, so careful
        result_df = final_df[~final_df[column].isin(new_df[column])]
        return pd.concat([result_df, new_df])


directory = 'test/'
salt = None
old_date, old_type = -1, None
final_df, final_events_df, final_locations_df = pd.DataFrame(), pd.DataFrame(), pd.DataFrame()

ugc_df = pd.read_csv('ugc_master.csv')
county_zip_df = pd.read_csv(
    'zip_code_database.csv',
    dtype = {
        'zip':str,
        'state':str,
        'ugc':str,
        'clean_county':str,
        'latitude':float,
        'longitude':float,
        'irs_estimated_population':int
        })
county_cleaner = (
    r"(North[ews]\w*|South[ews]\w*|Eastern|Western|Central|Interior|Coastal|"
    r"\s?Inland|\s?County|\s?Borough|Mountains?|Gorge|Area|Slopes|Upper|Lower)\s?")

file_matcher = (
    r"_(?P<year>20\d{2})"
    r"(?P<month>\d{2})"
    r"(?P<date>\d{2})"
    r"_(?P<hour>\d{2}_)"
    r"(?P<filetype>events|event_locations)"
)

for file in sorted(listdir(directory)):
    # Make sure it's a filename format we expect
    # The result of this wille a dict with keys [year,month,date,hour,filetype]
    try: 
        match_dict = search(file_matcher,file).groupdict()
        if int(match_dict['year']) != year or int(match_dict['month']) != month:
            print('Found a file outside of file format, skipping',file)
    except:
        print('Failed to parse file name, skipping',file)
        continue
    
    # ensure the files are sorted
    # fill_or_swap_duplicates assumes the more recently viewed file is newer, 
    # and therefore has the most up-to-date information
    # So this ends up being important 
    match_dict['date'] = int(match_dict['date'])
    
    if old_date < 0:
        old_date = match_dict['date'] - 1
        old_type = match_dict['filetype']
    
    if old_type != match_dict['filetype'] and old_date == match_dict['date']:
        old_type = match_dict['filetype']
    elif old_date > match_dict['date']:
        assert False, "Files are not sorted! Files must be sorted alphanumerically."
    
    old_date = match_dict['date']
    
    filepath = directory + file 
    if match_dict['filetype'] == 'events':
        event_file_df = pd.read_csv(filepath, 
                              parse_dates = [2,3], 
                              date_parser=lambda col: pd.to_datetime(col, utc=True))
        final_events_df = fill_or_swap_duplicates(final_events_df, event_file_df)
    elif match_dict['filetype'] == 'event_locations':
        locations_file_df = pd.read_csv(filepath)
        final_locations_df = fill_or_swap_duplicates(final_locations_df, locations_file_df)
    else:
        print("I dunno what to do with this:",filepath)

In [30]:
# There are differences between the zip-county database and the NWS' county identification 
# This reconciles most of them, the ones that aren't caught tend to be geographic-specific locations 
ugc_df['clean_county'] = ugc_df['name'].apply(lambda x: sub(county_cleaner,'',x).strip())

# Build up a ugc-zip data table based on the UGCs we know so far 
ugc_zip = ugc_df.merge(
    county_zip_df, 
    left_on=['state','clean_county'], 
    right_on=['state','clean_county'],
    how='left')

# Rename, filter out to just the columns we need
ugc_zip = ugc_zip.rename(columns={'ugc_x':'ugc'})[['ugc','name','state','clean_county','zip','irs_estimated_population']]

# Make a table that joins our event_locations to this zip database
loc_zip = final_locations_df.merge(
    ugc_zip,
    left_on='ugc',
    right_on='ugc')

# Pandas assumes nothing about datatypes, so let's convert some stuff
loc_zip['nws_id'] = loc_zip['nws_id'].astype(str)
loc_zip['zip'] = loc_zip['zip'].astype(str)
final_events_df['nws_id'] = final_events_df['nws_id'].astype(str)

#One more gigantic merge 
prep_final_df = final_events_df.merge(
    loc_zip, 
    on='nws_id')

# Filter out events we don't have a zip for (because they're probably mountains or a gorge or whatever)
# Then filter out lesser events if Severe is True, otherwise bring in everyone 
final_df = ( prep_final_df[(~prep_final_df['zip'].isna()) &
                           (True if not(severe) else prep_final_df['severity'].isin(['Severe','Extreme']))]
           )

# Final columns 
print_df = final_df[[
    'clean_county',
    'state',
    'zip',
    'start',
    'end',
    'severity',
    'type',
    'snow_min',
    'snow_max',
    'ice_min',
    'ice_max',
    'sleet_min',
    'sleet_max',  
]]

print_df.reset_index(drop=True).to_csv("data/reports/{}{}_adverse_weather_report.csv".format(year,month),index=False)
print_df

,clean_county,state,zip,start,end,severity,type,snow_min,snow_max,ice_min,ice_max,sleet_min,sleet_max
109908,Big Island Summit,HI,nan,2022-12-19 13:31:00+00:00,2022-12-20 16:00:00+00:00,Severe,Winter Storm Warning,0.0,8.0,NaN,NaN,NaN,NaN
109909,Big Island Summit,HI,nan,2022-12-19 13:31:00+00:00,2022-12-20 16:00:00+00:00,Severe,Winter Storm Warning,0.0,8.0,NaN,NaN,NaN,NaN
109910,Big Island Summit,HI,nan,2022-12-19 13:31:00+00:00,2022-12-20 16:00:00+00:00,Severe,Winter Storm Warning,0.0,8.0,NaN,NaN,NaN,NaN
109911,Big Island Summit,HI,nan,2022-12-19 13:31:00+00:00,2022-12-20 16:00:00+00:00,Severe,Winter Storm Warning,0.0,8.0,NaN,NaN,NaN,NaN
109912,Big Island Summit,HI,nan,2022-12-19 13:31:00+00:00,2022-12-20 16:00:00+00:00,Severe,Winter Storm Warning,0.0,8.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
484011,West Glacier Region,MT,nan,2022-12-19 19:52:00+00:00,2022-12-22 00:00:00+00:00,Severe,Winter Storm Warning,6.0,11.0,NaN,NaN,NaN,NaN
484012,Bitterroot/Sapphire,MT,nan,2022-12-19 19:52:00+00:00,2022-12-22 00:00:00+00:00,Severe,Winter Storm Warning,6.0,11.0,NaN,NaN,NaN,NaN
484013,Kootenai/Cabinet Region,MT,nan,2022-12-19 19:52:00+00:00,2022-12-22 00:00:00+00:00,Severe,Winter Storm Warning,5.0,11.0,NaN,NaN,NaN,NaN
484014,Butte/Blackfoot Region,MT,nan,2022-12-19 19:52:00+00:00,2022-12-22 00:00:00+00:00,Severe,Winter Storm Warning,4.0,6.0,NaN,NaN,NaN,NaN
